In [1]:
import pandas as pd
from plotly import express as px, graph_objects as go

## Lectura de CSV

In [2]:
df_producto=pd.read_csv('../data/EXPO_DOC_TOTAL.csv', 
                        encoding='Latin-1',
                        # sep=';', 
            dtype={'ncm':'str','CUIT':'str','empresa':'str','enmienda':'str','ncm_descri':'str','sim':'str','pais':'str','pais_descri':'str','dia':'str','mes':'str','anio':'str'}
            )

C:\Users\lgelmini\AppData\Local\Temp\ipykernel_28188\2993245896.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_producto=pd.read_csv('../data/EXPO_DOC_TOTAL.csv',


## Wrangling

In [3]:
# Correcciones de lectura
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize())
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower())

#Fob por tonelada importada
df_producto['fob_unitario_ton']=df_producto.fob/df_producto.pnet*1000

#Crear columna que una ncm con su descri
def recortar_descri_ncm(ncm_con_descri, caracteres=75):
    '''Recortar la descripcion de las ncm, como para que en el dropdown aparezcan bien. Sino puede ser muy largo'''
    if len(ncm_con_descri) > caracteres:
        return ncm_con_descri[:caracteres-3]+'...'
    else:
        return ncm_con_descri
    
df_producto['ncm_con_descri']=(df_producto.ncm.astype(str)+' - '+df_producto.ncm_descri).apply(recortar_descri_ncm)

# Fechas para usarlas en los graficos
desde=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'][0]
hasta=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'].iloc[-1]


inicio=df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes[0]+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio[0]
mes=int(df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes.iloc[-1])+1
fin=str(mes)+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio.iloc[-1]
monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=inicio, end=fin, freq='M').strftime('%m-%Y')
})

## Funciones filtro

In [4]:
def recortar_descri(pais,largo=15):
    if len(pais)<=largo:
        return pais
    else: return str(pais[:largo]+'...')
    
df_producto.pais_descri.apply(recortar_descri)

0                        Peru
1          Corea, republic...
2                       Japon
3                       India
4                        Peru
                  ...        
1087636                Brasil
1087637                Brasil
1087638                Brasil
1087639                Brasil
1087640                Brasil
Name: pais_descri, Length: 1087641, dtype: object

In [5]:
#Funcion para precio de referencia (endogeno)

def precio_ref(df):
    df_mensual=df.groupby(['mes','anio'],as_index=False).sum().sort_values(['anio','mes'])
    df_mensual['fob_unitario_ton_ref']=df_mensual.fob/df_mensual.pnet*1000
    df=pd.merge(left=df,right=df_mensual[['anio','mes','fob_unitario_ton_ref']], on=['anio','mes'],how='left')
    df['fob_unitario_ton_capit']=df.fob_unitario_ton*df.fob_unitario_ton_ref.iloc[-1]/df.fob_unitario_ton_ref
    df['diferencia_ref']=df.fob_unitario_ton-df.fob_unitario_ton_ref
    df['diferencia_ref_capi']=df.fob_unitario_ton_capit-df.fob_unitario_ton_ref.iloc[-1]
    return df

# Primer filtro está dado por la ncm

def ncm_filtro(df,ncm):
    df=df.loc[df.ncm==ncm].reset_index(drop=True)
    df['fecha']=df.mes+'-'+df.anio
    # df['fecha']=pd.to_datetime(df["fecha"]).dt.strftime('%m-%Y')
    df=df.sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
    return precio_ref(df)

def sim_unique(ncm):
    # return df.sim.unique()
    return df_producto[df_producto.ncm==ncm].sim.unique()

In [6]:
#Segundo filtro dado por el codigo sim. En caso de dafualt, no hacer filtro

def sim_filtro(df,ncm, sim='default', pais='default'):
    if sim=='default' and pais=='default':
        return ncm_filtro(df,ncm)
    elif sim=='default' and pais!='default':
        pais=pais.capitalize()
        return ncm_filtro(df,ncm)[ncm_filtro(df,ncm).pais_descri==pais].reset_index(drop=True)
    elif sim!='default' and pais=='default':
        return ncm_filtro(df,ncm)[ncm_filtro(df,ncm).sim==sim].reset_index(drop=True)
    else: 
        pais=pais.capitalize()
        return ncm_filtro(df,ncm)[(ncm_filtro(df,ncm).sim==sim)&(ncm_filtro(df,ncm).pais_descri==pais)].reset_index(drop=True)



# sim_filtro_2(ncm_filtro(df_producto,df_producto.ncm[0]),ncm_filtro(df_producto,df_producto.ncm[0]).sim[0])

In [7]:
def get_top_paises(df, n_paises='default'):
    if n_paises=='default': return df
    else:
        top_df=df.groupby(['pais','pais_descri'],as_index=False).sum().sort_values('fob',ascending=False).pais_descri.unique()[:n_paises]
        return df[df.pais_descri.isin(top_df)].reset_index(drop=True)
    
# get_top_paises(sim_filtro_2(ncm_filtro(df_producto,df_producto.ncm_con_descri[0]),ncm_filtro(df_producto,df_producto.ncm_con_descri[0]).sim[0]), 1)

In [8]:
df_producto.ncm_con_descri[0]

'22042100 - Vinos no espumosos, mosto de uva en el que la fermentacion se...'

In [9]:
def get_descri_nomen(df, palabras:int = 15):
    '''Entra lista de dataFrames y numero maximo de palabras'''
    for d in df.ncm_descri.unique():
        if type(d)!=float:
            if len(d.split(' '))>palabras:
                a=' '.join(d.split(' ')[:palabras])
                return a+'...'
            else: 
                return d

## Funciones plots

In [10]:
def precio_boxplot_capitalizado(df,ncm,sim='default', n_paises='default'):
     ncm=str(ncm)
     df=sim_filtro(df,ncm,sim)
     df=get_top_paises(df,n_paises)
     producto=df.ncm_descri.unique()[0][:40]
     ncm=df.ncm.unique()[0]
     if sim=='default' and n_paises=='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,9)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'     
     elif sim=='default' and n_paises!='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises} países'
     elif sim!='default' and n_paises=='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'
     elif sim!='default' and n_paises!='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises} países'
            
     precio_soja_boxplot=px.box(
     df.round(1),
     # x='fecha',
     x='fob_unitario_ton_capit',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob_unitario_ton_capit':'Precio capitalizado',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                 'fecha',
                    'fob_unitario_ton',
                    'fob_unitario_ton_capit',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[producto['fob_unitario_ton_capit'].min()-100,producto['fob_unitario_ton_capit'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=750,
                                   template = 'none',
                                   title=dict(text=title_text,
                                              y=0.95),
                                   # title_text=title_text,
                                   showlegend=False,
                                   margin=dict(t=150))
     
     precio_soja_boxplot.add_vline(x=df.fob_unitario_ton_ref.iloc[-1],line_dash="dash",line_color="blue")

     return precio_soja_boxplot

def plot_precio(df,ncm,sim='default', n_paises='default', color='pais_descri',pais='default'):
     '''color=pais_descri o empresa'''
     ncm=str(ncm)
     df=sim_filtro(df,ncm,sim,pais)
     df=get_top_paises(df,n_paises)
     df.pais_descri=df.pais_descri.apply(recortar_descri)
     df.empresa=df.empresa.apply(recortar_descri)
     producto=df.ncm_descri.unique()[0][:40]
     ncm=df.ncm.unique()[0]
     if sim=='default' and n_paises=='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,9)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'     
     elif sim=='default' and n_paises!='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises} países'
     elif sim!='default' and n_paises=='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'
     elif sim!='default' and n_paises!='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises} países'
                 
     precio_soja_plot=px.scatter(
     df.round(1),
     x='fecha',
     y='diferencia_ref',
     color=color,
     category_orders={"fecha": monthDates.fecha,
                      'empresa': df.empresa.sort_values(ascending=True),},
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_plot.update_yaxes(title_text= 'Diferencia en USD', 
                                   range=[df['diferencia_ref'].min()-100,df['diferencia_ref'].max()+100]
                                   )
     precio_soja_plot.update_xaxes(title_text='',type='category')

     precio_soja_plot.update_traces(marker=dict(size=12,
                                   line=dict(width=2,
                                             color='DarkSlateGrey')),
                    selector=dict(mode='markers'))

     precio_soja_plot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title=dict(text=title_text,
                                              y=0.95),
                                   # title_text=title_text,
                                   # showlegend=False,
                                   margin=dict(t=150))
                                   

     return precio_soja_plot


In [26]:
def precio_violinplot_capitalizado(df,ncm,sim='default', n_paises=5, max_range=None):
     ncm=str(ncm)
     df=sim_filtro(df,ncm,sim)
     df=get_top_paises(df,n_paises)
     producto=df.ncm_descri.unique()[0][:40]
     ncm=df.ncm.unique()[0]
     print(max_range)
     if sim=='default' and n_paises=='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,9)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'     
     elif sim=='default' and n_paises!='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises} países'
     elif sim!='default' and n_paises=='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'
     elif sim!='default' and n_paises!='default':
          title_text=f'FOB por tonelada exportada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises} países'
            
     if max_range:
          df = df[df.fob_unitario_ton_capit<max_range]
          
          
     # violinplot = go.Figure()
     # violinplot.add_trace(
     #      go.Violin(
     #           x = df.round(1)['pais_descri'],
     #           y = df.round(1)['fob_unitario_ton_capit']
     #      )
     # )
     
     violinplot=px.violin(
     data_frame = df.round(1),
     x='pais_descri',
     y='fob_unitario_ton_capit',
     box = True,
     points= 'all',
     color='pais_descri'
     )

     violinplot.update_yaxes(title_text= '', 
                                   # range=[producto['fob_unitario_ton_capit'].min()-100,producto['fob_unitario_ton_capit'].max()+100]
                                   )
     violinplot.update_xaxes(title_text='Fob unitario')

     violinplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=750,
                                   template = 'none',
                                   title=dict(text=title_text,
                                              y=0.95),
                                   # title_text=title_text,
                                   showlegend=False,
                                   margin=dict(t=150))
     
     # violinplot.add_hline(y=df.fob_unitario_ton_ref.iloc[-1],line_dash="dash",line_color="blue")

     return violinplot

## Outputs

In [13]:
sim_unique('12019000')

array(['190C', '110B', '299C', '219B'], dtype=object)

In [14]:
ncm='12024200'
precio_boxplot_capitalizado(df=df_producto,
                            ncm=ncm,
                            sim=sim_unique(ncm)[1])

plot_precio(df=df_producto,
            ncm=ncm,
            sim = sim_unique(ncm)[1],
            color='pais_descri',
            # pais='China',
            n_paises=5,
            )


In [15]:
precio_boxplot_capitalizado(df_producto,'12019000')

In [16]:
precio_violinplot_capitalizado(df_producto,'12019000',max_range = 10000)   

Esta es la funcion. Ncm 12019000, max_range: 10000


In [34]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.Input(value='12019000', id='ncm'),
    html.Div(id='rango'),
    dcc.Slider(min=1, max=6, step=0.01,  id='max_range',
               value = 6,
               marks={i: '{}'.format(10 ** i) for i in range(7)},
               tooltip={"placement": "bottom", "always_visible": True},
               updatemode='drag'),
    dcc.Graph(id='violines')
        
  
],style={'backgroundColor': 'white'})

@app.callback(Output('rango', 'children'), Input('max_range', 'value'))
def rango(r):
    return dcc.Markdown(f'{10**r}')

@app.callback(
    Output('violines', 'figure'),
    Input('max_range', 'value'), Input('ncm', 'value')
)
def grafico(max_range, ncm):
    max_range = 10 **max_range # Lo convierto a log
    return precio_violinplot_capitalizado(df_producto,str(ncm),max_range = max_range)   
    
app.run_server(mode='inline')


1000000
977237.2209558112
8709.635899560806
1995.2623149688789
489.77881936844614
128.82495516931337
48.97788193684461
41.68693834703355
32.359365692962825
31.622776601683793
30.19951720402016
28.183829312644534
25.118864315095795
23.9883291901949
23.442288153199225
22.3872113856834
21.87761623949553
22.3872113856834
22.908676527677734
24.547089156850298
23.9883291901949
25.703957827688633
26.915348039269155
27.542287033381662
28.84031503126606
28.183829312644534
29.512092266663856
30.902954325135905
38.90451449942807
69.18309709189366
47.86300923226383
70.79457843841381
72.44359600749902
74.13102413009177
87.09635899560806
79.43282347242814
97.72372209558107104.71285480508996

114.81536214968828
120.22644346174131
123.02687708123811
138.03842646028852
147.91083881682073
131.82567385564073
158.48931924611142
173.78008287493762
177.82794100389228
186.20871366628674
190.54607179632464
194.98445997580455
199.52623149688787
269.1534803926917
218.77616239495518
295.1209226666387
316.2277660

La línea punteada representa al precio de referencia

In [ ]:
precio_boxplot_capitalizado(df_producto,'12019000').write_html('../output/expo soja boxplot.html')